In [1]:
from topology import generate_topology, args
from placement_strategies import (
    LocalPlacementStrategy,
    RandomReplicaPlacementStrategy,
    PCenterPlacementStrategy,
LatencyAwarePlacementStrategy
)
from environment import DataContinuumEnv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns


# %%

In [2]:
#Generate Topology
topology, deployment_info, instance_manager = generate_topology(**args)

print(f"\nTopology Statistics:")
print(f"  Total nodes: {len(topology.nodes)}")
print(f"  Total edges: {len(topology.edges)}")

storage_nodes = [n for n, d in topology.nodes(data=True) if d.get('type') in ['cloud', 'fog', 'edge']]
terminal_nodes = [n for n, d in topology.nodes(data=True) if d.get('type') == 'terminal']

print(f"  Storage nodes: {len(storage_nodes)}")
print(f"    - Cloud: {len([n for n, d in topology.nodes(data=True) if d.get('type') == 'cloud'])}")
print(f"    - Fog: {len([n for n, d in topology.nodes(data=True) if d.get('type') == 'fog'])}")
print(f"    - Edge: {len([n for n, d in topology.nodes(data=True) if d.get('type') == 'edge'])}")
print(f"  Terminal nodes: {len(terminal_nodes)}")

print(f"\nApplication Instances:")
print(f"  Generator instances: {len(instance_manager.generator_instances)}")
print(f"  Consumer instances: {len(instance_manager.consumer_instances)}")


strong_consumers = sum(1 for cid in instance_manager.consumer_instances
                      if instance_manager.get_instance(cid)['consistency'] == 1)
weak_consumers = len(instance_manager.consumer_instances) - strong_consumers

print(f"\nConsistency Requirements:")
print(f"  Strong consistency consumers: {strong_consumers}")
print(f"  Weak consistency consumers: {weak_consumers}")






Topology Statistics:
  Total nodes: 330
  Total edges: 329
  Storage nodes: 19
    - Cloud: 1
    - Fog: 4
    - Edge: 14
  Terminal nodes: 311

Application Instances:
  Generator instances: 64
  Consumer instances: 36

Consistency Requirements:
  Strong consistency consumers: 4
  Weak consistency consumers: 32


In [3]:
#Create Environments for each strategy
config = {
    'enable_congestion': False,
    'enable_consistency': True,
    'random_state': 56,
    'max_steps': 1000,
    'time_step_duration': 1
}




env_local = DataContinuumEnv(
    topology=topology,
    instance_manager=instance_manager,
    placement_strategy=LocalPlacementStrategy(),
    **config
)
print(f"   Strategy name: {env_local.strategy_name}")

# Strategy 2: Random 3 replicas
print("\n2. Creating RANDOM strategy environment...")
env_random = DataContinuumEnv(
    topology=topology,
    instance_manager=instance_manager,
    placement_strategy=RandomReplicaPlacementStrategy(num_replicas=3),
    **config
)
print(f"   Strategy name: {env_random.strategy_name}")

# Strategy 3: P-Center 3 replicas
print("\n3. Creating P-CENTER strategy environment...")
env_pcenter = DataContinuumEnv(
    topology=topology,
    instance_manager=instance_manager,
    placement_strategy=PCenterPlacementStrategy(num_replicas=3),
    **config
)


   Strategy name: local_storage

2. Creating RANDOM strategy environment...
   Strategy name: random_3_replicas

3. Creating P-CENTER strategy environment...


In [4]:
# Run Strategies
# %% Run Local strategy

results_local = env_local.run(verbose=True)
results_random = env_random.run(verbose=True)
results_pcenter = env_pcenter.run(verbose=True)


Running simulation for strategy: local_storage
  Steps: 1000
  Congestion: False
  Consistency: True
Running simulation for strategy: random_3_replicas
  Steps: 1000
  Congestion: False
  Consistency: True
Running simulation for strategy: p_center_3_replicas
  Steps: 1000
  Congestion: False
  Consistency: True


In [5]:
print("\n" + "=" * 80)
print("LOCAL STRATEGY - DETAILED RESULTS")
print("=" * 80)
env_local.evaluation.print_results(results_local)

# %% Random results
print("\n" + "=" * 80)
print("RANDOM STRATEGY - DETAILED RESULTS")
print("=" * 80)
env_random.evaluation.print_results(results_random)

# %% P-Center results
print("\n" + "=" * 80)
print("P-CENTER STRATEGY - DETAILED RESULTS")
print("=" * 80)
env_pcenter.evaluation.print_results(results_pcenter)


LOCAL STRATEGY - DETAILED RESULTS

=== EVALUATION RESULTS ===
Data generated: 2593
Data consumed: 351
Storage usage: 0.00% (avg)
Network - Avg Congestion: 0.000
Total bytes transferred: 1,103,709,774 bytes

--- Strategy: local_storage ---
Average latency: 151.32 ms
Median latency: 75.96 ms
P95 latency: 471.20 ms
SLO violation rate: 86.89%
Total consumptions: 351

RANDOM STRATEGY - DETAILED RESULTS

=== EVALUATION RESULTS ===
Data generated: 2593
Data consumed: 351
Storage usage: 0.01% (avg)
Network - Avg Congestion: 0.000
Total bytes transferred: 1,443,287,902 bytes

--- Strategy: random_3_replicas ---
Average latency: 118.91 ms
Median latency: 62.49 ms
P95 latency: 359.99 ms
SLO violation rate: 74.69%
Total consumptions: 320

  Consistency Breakdown:
  STRONG (read all replicas):
    Avg latency: 92.74 ms
    SLO violations: 100.00%
    Consumptions: 89
  WEAK (read closest):
    Avg latency: 128.99 ms
    SLO violations: 64.94%
    Consumptions: 231

P-CENTER STRATEGY - DETAILED RES

In [6]:
#Comparaison
# %% Extract metrics for comparison
def extract_metrics(results, strategy_name):
    """Extract key metrics from results"""
    strategy_results = list(results['strategies'].values())[0]

    metrics = {
        'strategy': strategy_name,
        'avg_latency': strategy_results['average_latency'],
        'median_latency': strategy_results['median_latency'],
        'p95_latency': strategy_results['p95_latency'],
        'slo_violation_rate': strategy_results['slo_violation_rate'],
        'total_consumptions': strategy_results['total_consumptions'],
        'storage_usage': results['storage']['average_usage_percentage'],
        'avg_congestion': results['network']['average_congestion'],
        'max_congestion': results['network']['max_congestion'],
        'total_data_gb': results['network']['total_bytes_transferred'] / (1024**3),
        'data_generated': results['data_throughput']['generated'],
        'data_consumed': results['data_throughput']['consumed']
    }

    return metrics


In [7]:
# Create comparison dataframe
comparison_data = [
    extract_metrics(results_local, 'Local'),
    extract_metrics(results_random, 'Random'),
    extract_metrics(results_pcenter, 'P-Center')
]

df_comparison = pd.DataFrame(comparison_data)

# %% Display comparison table
print("\n" + "=" * 80)
print("STRATEGY COMPARISON TABLE")
print("=" * 80)
print("\nLatency Metrics:")
print(df_comparison[['strategy', 'avg_latency', 'median_latency', 'p95_latency']].to_string(index=False))

print("\nSLO Compliance:")
print(df_comparison[['strategy', 'total_consumptions', 'slo_violation_rate']].to_string(index=False))

print("\nResource Usage:")
print(df_comparison[['strategy', 'storage_usage', 'avg_congestion', 'total_data_gb']].to_string(index=False))


STRATEGY COMPARISON TABLE

Latency Metrics:
strategy  avg_latency  median_latency  p95_latency
   Local   151.319531       75.964000   471.196733
  Random   118.905675       62.489310   359.986106
P-Center    70.922405       35.390378   317.423688

SLO Compliance:
strategy  total_consumptions  slo_violation_rate
   Local                 351            0.868946
  Random                 320            0.746875
P-Center                 351            0.544160

Resource Usage:
strategy  storage_usage  avg_congestion  total_data_gb
   Local       0.004235             0.0       1.027910
  Random       0.012705             0.0       1.344167
P-Center       0.012705             0.0       1.447125
